# DSA Deep Learning [3] - Hypertuning Our CNN

In [1]:
# Import statements
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, ModelCheckpoint
from tensorflow.keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn.model_selection import train_test_split
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image, UnidentifiedImageError
import io
import base64
from google.colab import files
from google.colab.patches import cv2_imshow
from IPython.display import display, HTML, Javascript
from google.colab import output, files
import zipfile

In [ ]:
# Define paths to your CSV files
train_csv_path = 'fer2013.csv'
val_csv_path = 'train.csv'

# Display the first few rows of the CSV file to check column names
train_df = pd.read_csv(train_csv_path)
print(train_df.head())
print(train_df.columns)  # Print column names to verify

# Display the first few rows of the CSV file to check column names
val_df = pd.read_csv(val_csv_path)
print(val_df.head())
print(val_df.columns)  # Print column names to verify


   emotion                                             pixels     Usage
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...  Training
1        0  151 150 147 155 148 133 111 140 170 174 182 15...  Training
2        2  231 212 156 164 174 138 161 173 182 200 106 38...  Training
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...  Training
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...  Training
Index(['emotion', 'pixels', 'Usage'], dtype='object')
   emotion                                             pixels
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...
Index(['emotion', 'pixels'], dtype='object')


In [ ]:
# Load CSV data
def load_data(csv_path):
    df = pd.read_csv(csv_path)
    X = np.array([np.fromstring(pixel_str, dtype=float, sep=' ') for pixel_str in df['pixels']])
    X = X.reshape(-1, 48, 48, 1)  # Reshape to (n_samples, 48, 48, 1) for grayscale
    X = np.repeat(X, 3, axis=-1)  # Convert grayscale to RGB format by repeating channels
    X = X / 255.0  # Normalize pixel values
    y = to_categorical(df['emotion'])  # One-hot encode the labels
    return X, y

# Load training and validation data
X_train, y_train = load_data(train_csv_path)
X_val, y_val = load_data(val_csv_path)

In [ ]:
# Create a TensorFlow data generator
def create_generator(X, y, batch_size=64, shuffle=True):
    dataset = tf.data.Dataset.from_tensor_slices((X, y))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=1024)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)
    return dataset

# Create training and validation generators
batch_size = 64
train_generator = create_generator(X_train, y_train, batch_size=batch_size, shuffle=True)
val_generator = create_generator(X_val, y_val, batch_size=batch_size, shuffle=False)


In [ ]:
# Decode one-hot encoded labels in `y_train` back to single integers
y_train_labels = np.argmax(y_train, axis=1)

# Calculate class weights to handle class imbalance
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train_labels),
    y=y_train_labels
)
class_weights_dict = dict(enumerate(class_weights))

# Data augmentation (only for training set)
train_datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)


In [ ]:
# Load pre-trained MobileNetV2 and add custom layers
base_model = MobileNetV2(input_shape=(48, 48, 3), include_top=False, weights='imagenet')

# Freeze all layers initially
for layer in base_model.layers:
    layer.trainable = False

# Add custom layers on top
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu', kernel_regularizer=l2(0.01))(x)  # L2 regularization
x = Dropout(0.5)(x)  # Dropout layer to reduce overfitting
predictions = Dense(7, activation='softmax')(x)  # Output layer with 7 emotion classes

model = Model(inputs=base_model.input, outputs=predictions)

# Fine-tune the top layers
for layer in base_model.layers[-20:]:
    layer.trainable = True

# Compile the model
model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])

<ipython-input-9-5deb769f83ec>:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = MobileNetV2(input_shape=(48, 48, 3), include_top=False, weights='imagenet')


In [ ]:
# Callback to reduce learning rate when validation loss plateaus
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-5)

# Early stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Save the best model during training
checkpoint = ModelCheckpoint('emotion_recognition_model_advanced.keras', monitor='val_loss', save_best_only=True)

# List of callbacks
callbacks = [reduce_lr, early_stopping, checkpoint]


In [ ]:
# Train the model
history = model.fit(
    train_generator,
    epochs=20,                           # Adjust based on your needs
    validation_data=val_generator,
    class_weight=class_weights_dict,
    callbacks=callbacks
)


Epoch 1/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 43s 41ms/step - accuracy: 0.2066 - loss: 4.3287 - val_accuracy: 0.3141 - val_loss: 3.5972 - learning_rate: 1.0000e-04
Epoch 2/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step - accuracy: 0.3220 - loss: 3.2666 - val_accuracy: 0.3832 - val_loss: 2.8432 - learning_rate: 1.0000e-04
Epoch 3/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.3748 - loss: 2.6462 - val_accuracy: 0.4509 - val_loss: 2.3171 - learning_rate: 1.0000e-04
Epoch 4/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 12s 22ms/step - accuracy: 0.4195 - loss: 2.1819 - val_accuracy: 0.5156 - val_loss: 1.9421 - learning_rate: 1.0000e-04
Epoch 5/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 20s 20ms/step - accuracy: 0.4568 - loss: 1.8753 - val_accuracy: 0.5463 - val_loss: 1.6989 - learning_rate: 1.0000e-04
Epoch 6/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 9s 16ms/step - accuracy: 0.4945 - loss: 1.6476 - val_accuracy: 0.5785 - val_loss: 1.5021 - learning_rate: 1.0000e-04
Epoch 7/20
561/561 ━━━━━━━━━━━━━━━━━━━━ 10s 18ms/step 

In [2]:
# Path to the best saved model
model_path = 'emotion_recognition_model_advanced.keras'

# Load the best saved model if it exists
if os.path.exists(model_path):
    print("Loading the best saved model for predictions...")
    model = load_model(model_path)
    print("Model loaded successfully.")
else:
    print("No saved model found. Please train the model first.")


Loading the best saved model for predictions...
Model loaded successfully.


In [3]:
# Define a dictionary for emotion labels based on FER2013 class order
emotion_labels = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral"
}

# Updated predict_emotion function to handle multiple faces
def predict_emotion(frame, model):
    # Convert to grayscale for face detection
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

    # Load the face detection model (Haar Cascade)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect multiple faces in the frame
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(48, 48))

    # Process each detected face
    for (x, y, w, h) in faces:
        # Extract the face region from the frame
        face = frame[y:y+h, x:x+w]

        # Resize face region to 48x48, the input size expected by the model
        face_resized = cv2.resize(face, (48, 48))

        # Preprocess face (normalize and add batch dimension)
        face_array = np.expand_dims(face_resized, axis=0) / 255.0  # Scale pixel values to [0, 1]

        # Predict emotion
        emotion_prediction = model.predict(face_array)
        emotion = np.argmax(emotion_prediction)  # Get the emotion class with the highest probability

        # Draw a circle around the face and add the emotion label
        cv2.rectangle(frame, (x, y), (x+w, y+h), (255, 0, 0), 2)
        emotion_label = emotion_labels[emotion]  # Map the predicted emotion index to label
        cv2.putText(frame, emotion_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame


In [ ]:
# JavaScript code to start the live webcam feed and capture image upon button click
def start_webcam_feed():
    js = """
    <script>
        let videoElement = null;
        let stream = null;

        async function startVideo() {
            if (!videoElement) {
                videoElement = document.createElement('video');
                videoElement.setAttribute('autoplay', '');
                videoElement.setAttribute('playsinline', '');
                document.body.appendChild(videoElement);
                stream = await navigator.mediaDevices.getUserMedia({ video: true })
                  .catch(err => {
                      console.error('Webcam not accessible:', err);
                      alert('Webcam not accessible. You can upload an image instead.');
                  });
                if (stream) {
                    videoElement.srcObject = stream;
                }
            }
        }

        async function capturePhoto() {
            if (!videoElement) {
                alert("Webcam is not active!");
                return;
            }
            const canvas = document.createElement('canvas');
            canvas.width = videoElement.videoWidth;
            canvas.height = videoElement.videoHeight;
            canvas.getContext('2d').drawImage(videoElement, 0, 0);

            // Stop video feed
            stream.getTracks().forEach(track => track.stop());
            videoElement.remove();
            videoElement = null;

            // Convert the photo to base64 and send to Python
            const dataUrl = canvas.toDataURL('image/jpeg');
            google.colab.kernel.invokeFunction('notebook.get_webcam_image', [dataUrl], {});
        }

        // Add the start and capture buttons to the DOM
        const startButton = document.createElement('button');
        startButton.innerHTML = 'Start Webcam Feed';
        startButton.onclick = startVideo;
        document.body.appendChild(startButton);

        const captureButton = document.createElement('button');
        captureButton.innerHTML = 'Capture Photo';
        captureButton.onclick = capturePhoto;
        document.body.appendChild(captureButton);
    </script>
    """
    display(HTML(js))

# Callback function to receive the captured image in Python
def get_webcam_image(dataUrl):
    img_data = base64.b64decode(dataUrl.split(",")[1])
    img = Image.open(io.BytesIO(img_data))
    img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
    processed_img = predict_emotion(img, model)
    cv2_imshow(processed_img)

# Function to enable file upload
def enable_file_upload():
    from google.colab import files
    print("Upload an image if you prefer.")
    uploaded = files.upload()

    for file_name in uploaded.keys():
        img = Image.open(file_name)
        img = cv2.cvtColor(np.array(img), cv2.COLOR_RGB2BGR)
        processed_img = predict_emotion(img, model)
        cv2_imshow(processed_img)

# Display the file upload button
def display_file_upload_button():
    js = """
    <script>
        const uploadButton = document.createElement('button');
        uploadButton.innerHTML = 'Upload an Image';
        uploadButton.onclick = () => {
            google.colab.kernel.invokeFunction('notebook.enable_file_upload', [], {});
        };
        document.body.appendChild(uploadButton);
    </script>
    """
    display(HTML(js))

# Register the callbacks
output.register_callback('notebook.get_webcam_image', get_webcam_image)
output.register_callback('notebook.enable_file_upload', enable_file_upload)

# Initialize the webcam feed, buttons, and file upload option
start_webcam_feed()
display_file_upload_button()
